`NOTE: this is using decontextualized sentence gpt2-xl surprisals!`
    
`NOTE: using gpt2 surprisal averaged by number of tokens in the string!`

In [738]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import copy
import json

In [739]:
import plot_utils
import stats_utils

In [740]:
save = True

In [741]:
average_surprisals = "NrTokens"
exclude_randomwl = False
exclude_concatenated_control = True

In [742]:
model_identifier = "gpt2-xl"
randomnouns=False

In [743]:
#load configuration
with open('configurations.json', 'r') as f:
    config = json.load(f)
    
list(config.keys())

['TrainIntact-TestPerturbed:contextualized',
 'TrainIntact-TestPerturbed:decontextualized',
 'TrainPerturbed-TestPerturbed:contextualized',
 'TrainPerturbed-TestPerturbed:decontextualized',
 'TrainPerturbed-TestPerturbed:cv-by-passage']

In [744]:
# get setting
approach = "TrainIntact-TestPerturbed:contextualized"

emb_context=config[approach]["emb_context"]
split_coord=config[approach]["split_coord"]
testonperturbed=config[approach]["testonperturbed"]

In [745]:
if testonperturbed:
    to_prepend = "teston:"
else:
    to_prepend = ""

# Get brainscores

In [746]:
def flatten_list(xss):
    return [x for xs in xss for x in xs]

In [747]:
def get_conditions(testonperturbed=False, randomnouns=False):
    
    if testonperturbed:
        to_prepend = "teston:"
    else:
        to_prepend = ""
        
    original = [f'{to_prepend}original']
    
    conditions_control = [f'{to_prepend}random-wl',
                         f'{to_prepend}concatenated-control']

    conditions_scrambled = [f'{to_prepend}scrambled1',
                            f'{to_prepend}scrambled3',
                            f'{to_prepend}scrambled5',
                            f'{to_prepend}scrambled7',
                            f'{to_prepend}backward',
                            f'{to_prepend}lowpmi',
                            f'{to_prepend}lowpmi-random']
    
    if testonperturbed:
        conditions_scrambled = [re.sub("scrambled","scr",elm) for elm in conditions_scrambled]

    conditions_perturb_loss = [f'{to_prepend}nouns',
                               f'{to_prepend}nounsverbs',
                               f'{to_prepend}nounsverbsadj',
                               f'{to_prepend}contentwords',
                               f'{to_prepend}functionwords']
    
    if randomnouns:
        conditions_perturb_loss += [f'{to_prepend}random-nouns']

    conditions_perturb_meaning = [f'{to_prepend}sentenceshuffle_passage',
                                  f'{to_prepend}sentenceshuffle_topic',
                                  f'{to_prepend}sentenceshuffle_random']


    #create CAT2COND
    conditions = [original, conditions_scrambled, conditions_perturb_loss, conditions_perturb_meaning, conditions_control]
    categories_unique = ["original", "word-order", "information-loss", "semantic-distance", "control"]
    
    CAT2COND = dict(zip(categories_unique, conditions)) #dictionary from manipulation category to condition
    
    #create COND2CAT
    categories = [["original"] * len(original),
                  ["word-order"] * len(conditions_scrambled),
                  ["information-loss"] * len(conditions_perturb_loss),
                  ["semantic-distance"] * len(conditions_perturb_meaning),
                  ["control"] * len(conditions_control)]
    
    conditions = flatten_list(conditions)
    categories = flatten_list(categories)

    COND2CAT = dict(zip(conditions, categories))

    return CAT2COND, COND2CAT

In [748]:
def get_max_score(matrix):
    """
    input: result = out['data'].values matrix (e.g. for distilgpt2 a matrix of dimensions 7x2)
    output: maximum score and associated error for this matrix.
    """
    max_score, error = 0,0
    for i in range(len(matrix)):
        if matrix[i][0] > max_score:
            max_score = matrix[i][0]
            error = matrix[i][1]
    return max_score, error

In [749]:
#function check
def get_best_scores_df(model_identifier, emb_context="Passage", split_coord="Sentence", testonperturbed=False, randomnouns=False):
    """
    input: model_identifier, embedding context, split_coordinate & whether to test on perturbed sentence
    output: dataframe containing the maximum score and associated error per condition.
    """
    
    working_dir = "/om2/user/ckauf/.result_caching/neural_nlp.score"
    
    CAT2COND, COND2CAT = get_conditions(testonperturbed=testonperturbed, randomnouns=randomnouns)
    
    conditions, categories = [], []
    max_scores, errors = [], []
    
    for filename in os.listdir(working_dir):
        
        if os.path.isdir(os.path.join(working_dir,filename)):
            continue
            
        if not testonperturbed:
            if "teston:" in filename:
                continue
        else:
            if not "teston:" in filename:
                continue
                
        if not f"emb_context={emb_context}" in filename:
            continue
                
        if not f"split_coord={split_coord}" in filename:
            continue
        
        exclude_list = ["constant-control", "length-control", "random-nouns"]
        if randomnouns:
            exclude_list = ["constant-control", "length-control"]
            
        if any(x in filename for x in exclude_list):
            continue
                        

        model_name = filename.split(",")[1]
        bm = filename.split(",")[0]
        
        if bm == "benchmark=Pereira2018-encoding": #exclude old orignial bm in different format
            continue
        
        if "model=" + model_identifier == model_name:
            
            condition = bm.split("benchmark=Pereira2018-encoding-")[-1]
                
#             print(filename, sys.stdout.flush())
            
            #clean name
            condition = re.sub("perturb-","",condition)
            if not any(x in condition for x in ["1", "3", "5", "7"]):
                condition = re.sub("scrambled-","",condition)
            
            if testonperturbed:
                condition = re.sub("scrambled","scr",condition)
                
#             print(condition)

            #load scores
            file = os.path.join(working_dir,filename)
            with open(file, 'rb') as f:
                out = pickle.load(f)
#                 lang_score_matrix = get_lang_score(out)
#                 max_score, error = get_max_score(lang_score_matrix)
                result = out['data'].values
#                 #print(result, '\n\n')
                max_score, error = get_max_score(result)

            conditions.append(condition)
            categories.append(COND2CAT[condition])
            max_scores.append(max_score)
            errors.append(error)

                
    import pandas as pd
    
    index = conditions
    condition_order = list(COND2CAT.keys())
    
    df = pd.DataFrame({
        'score': max_scores,
        'error': errors,
        'condition': conditions,
        'category': categories})
    
    df['condition'] = pd.Categorical(df['condition'], categories=condition_order, ordered=True)
    scores_df = df.sort_values(by='condition').reset_index(drop=True)
    
    #clean names
    scores_df['condition'] = scores_df['condition'].str.replace("teston:","")
    scores_df['condition'] = scores_df['condition'].replace(
    {'sentenceshuffle_random': 'sent_random',
    'sentenceshuffle_passage': 'sent_passage',
    'sentenceshuffle_topic': 'sent_topic',
    'scr1': 'scrambled1',
    'scr3': 'scrambled3',
    'scr5': 'scrambled5',
    'scr7': 'scrambled7'}
    )
    
    return scores_df

scores_df = get_best_scores_df(model_identifier=model_identifier,
                               emb_context=emb_context,
                               split_coord=split_coord,
                               testonperturbed=testonperturbed,
                               randomnouns=randomnouns)



In [750]:
brainscores = list(scores_df["score"])
delta_brainscores = [brainscores[i] - brainscores[0] for i in range(len(brainscores))]
scores_df["delta_brainscore"] = delta_brainscores
scores_df

,score,error,condition,category,delta_brainscore
0,1.107199,0.141840,original,original,0.000000
1,1.019374,0.120393,scrambled1,word-order,-0.087825
2,0.924364,0.086006,scrambled3,word-order,-0.182834
3,0.907545,0.071722,scrambled5,word-order,-0.199654
4,0.914475,0.099400,scrambled7,word-order,-0.192724
5,0.835083,0.064965,backward,word-order,-0.272116
6,0.933212,0.070269,lowpmi,word-order,-0.173986
7,0.889217,0.093797,lowpmi-random,word-order,-0.217982
8,0.644818,0.078112,nouns,information-loss,-0.462381
9,0.808808,0.098496,nounsverbs,information-loss,-0.298390


# Load surprisals

In [751]:
surprisal_path = os.path.abspath("./gpt2_surprisals/files")

#distilgpt2.perturbed+contentwords.sentence_surp.average.txt
#Index	Sentence	Surprisal	PPL

In [752]:
COND2SET = {
'Original' : 'original',
    #
'Scr1' : 'scrambled',
'Scr3' : 'scrambled',
'Scr5' : 'scrambled',
'Scr7' : 'scrambled',
'backward' : 'scrambled',
'lowPMI' : 'scrambled',
'random' : 'scrambled',
'nouns' : 'perturbed',
'randomnouns' : 'perturbed',
'nounsverbs' : 'perturbed',
'nounsverbsadj' : 'perturbed',
'contentwords' : 'perturbed',
'functionwords' : 'perturbed',
    #
'sentenceshuffle-random' : 'perturbed',
'sentenceshuffle-withinpassage' : 'perturbed',
'sentenceshuffle-withintopic' : 'perturbed',
'concatenated_control' : 'control'
}

In [753]:
CONDMAP = [ #same name between brainscores and surprisals
('Original', f'{to_prepend}original'),
    #
('Scr1', f'{to_prepend}scr1'),
('Scr3', f'{to_prepend}scr3'),
('Scr5', f'{to_prepend}scr5'),
('Scr7', f'{to_prepend}scr7'),
('backward', f'{to_prepend}backward'),
('lowPMI', f'{to_prepend}lowpmi'),
('random', f'{to_prepend}random-wl'),
('nouns', f'{to_prepend}nouns'),
('randomnouns', f'{to_prepend}random-nouns'),
('nounsverbs', f'{to_prepend}nounsverbs'),
('nounsverbsadj', f'{to_prepend}nounsverbsadj'),
('contentwords', f'{to_prepend}contentwords'),
('functionwords', f'{to_prepend}functionwords'),
('concatenated_control', f'{to_prepend}concatenated-control')
]

In [754]:
conditions, categories, avg_surprisal = [], [], []
for (cond, category) in CONDMAP:
    scr_perturb = COND2SET[cond]
    if average_surprisals == "NrTokens":
        filename = f"{model_identifier}.{scr_perturb}+{cond}.sentence_surp.average.txt"
    elif average_surprisals == "NrWords":
        filename = f"{model_identifier}.{scr_perturb}+{cond}.sentence_surp.average_sentlength.txt"
    else:
        break
    try:
        content = pd.read_csv(os.path.join(surprisal_path, filename), sep="\t")
    except:
        print(f"not computed for cond {cond}!")
        continue
    avg_surprisal.append(content["Surprisal"].mean())
    conditions.append(category)
    categories.append(scr_perturb)

surprisal_df = pd.DataFrame({
    "condition" : conditions,
    "category" : categories,
    "surprisal" : avg_surprisal
})

delta_surprisal = [avg_surprisal[0] - avg_surprisal[i] for i in range(len(conditions))]

surprisal_df["delta_surprisal"] = delta_surprisal
surprisal_df

,condition,category,surprisal,delta_surprisal
0,teston:original,original,4.355824,0.000000
1,teston:scr1,scrambled,5.287545,-0.931720
2,teston:scr3,scrambled,5.970837,-1.615013
3,teston:scr5,scrambled,6.324255,-1.968431
4,teston:scr7,scrambled,7.095926,-2.740101
5,teston:backward,scrambled,7.501000,-3.145176
6,teston:lowpmi,scrambled,7.492144,-3.136320
7,teston:random-wl,scrambled,8.264668,-3.908843
8,teston:nouns,perturbed,7.902859,-3.547035
9,teston:random-nouns,perturbed,8.964838,-4.609013


In [755]:
#clean names
surprisal_df['condition'] = surprisal_df['condition'].str.replace("teston:","")
surprisal_df['condition'] = surprisal_df['condition'].replace(
{
'scr1': 'scrambled1',
'scr3': 'scrambled3',
'scr5': 'scrambled5',
'scr7': 'scrambled7'}
)

surprisal_df = surprisal_df.drop(columns=['category'])
surprisal_df

,condition,surprisal,delta_surprisal
0,original,4.355824,0.000000
1,scrambled1,5.287545,-0.931720
2,scrambled3,5.970837,-1.615013
3,scrambled5,6.324255,-1.968431
4,scrambled7,7.095926,-2.740101
5,backward,7.501000,-3.145176
6,lowpmi,7.492144,-3.136320
7,random-wl,8.264668,-3.908843
8,nouns,7.902859,-3.547035
9,random-nouns,8.964838,-4.609013


# Set up plot df

In [756]:
plot_df = surprisal_df.merge(scores_df, how='inner', on='condition')
plot_df

,condition,surprisal,delta_surprisal,score,error,category,delta_brainscore
0,original,4.355824,0.000000,1.107199,0.141840,original,0.000000
1,scrambled1,5.287545,-0.931720,1.019374,0.120393,word-order,-0.087825
2,scrambled3,5.970837,-1.615013,0.924364,0.086006,word-order,-0.182834
3,scrambled5,6.324255,-1.968431,0.907545,0.071722,word-order,-0.199654
4,scrambled7,7.095926,-2.740101,0.914475,0.099400,word-order,-0.192724
5,backward,7.501000,-3.145176,0.835083,0.064965,word-order,-0.272116
6,lowpmi,7.492144,-3.136320,0.933212,0.070269,word-order,-0.173986
7,random-wl,8.264668,-3.908843,0.108709,0.070373,control,-0.998490
8,nouns,7.902859,-3.547035,0.644818,0.078112,information-loss,-0.462381
9,nounsverbs,7.467610,-3.111786,0.808808,0.098496,information-loss,-0.298390


In [757]:
# rename conditions according to new names:
# if testonperturbed:
#     plot_df['condition'] = [re.sub("teston:","",elm) for elm in list(plot_df['condition'])]
#     plot_df['condition'] = [re.sub("scr","scrambled",elm) for elm in list(plot_df['condition'])]                      
plot_df['condition'] = plot_df['condition'].map(plot_utils.COND2LABEL)
plot_df

,condition,surprisal,delta_surprisal,score,error,category,delta_brainscore
0,Original,4.355824,0.000000,1.107199,0.141840,original,0.000000
1,1LocalWordSwap,5.287545,-0.931720,1.019374,0.120393,word-order,-0.087825
2,3LocalWordSwaps,5.970837,-1.615013,0.924364,0.086006,word-order,-0.182834
3,5LocalWordSwaps,6.324255,-1.968431,0.907545,0.071722,word-order,-0.199654
4,7LocalWordSwaps,7.095926,-2.740101,0.914475,0.099400,word-order,-0.192724
5,ReverseOrder,7.501000,-3.145176,0.835083,0.064965,word-order,-0.272116
6,LowPMI,7.492144,-3.136320,0.933212,0.070269,word-order,-0.173986
7,RandWordList,8.264668,-3.908843,0.108709,0.070373,control,-0.998490
8,Nouns,7.902859,-3.547035,0.644818,0.078112,information-loss,-0.462381
9,NounsVerbs,7.467610,-3.111786,0.808808,0.098496,information-loss,-0.298390


In [758]:
if exclude_randomwl:
    plot_df = plot_df[plot_df.condition != "RandWordList"]
    
if exclude_concatenated_control:
    plot_df = plot_df[plot_df.condition != "ConcatenatedControl"]

In [759]:
f"Correlation: {plot_df['delta_surprisal'].corr(plot_df['delta_brainscore'])}"

'Correlation: 0.5767630072347961'

In [760]:
import plotly.express as px
import scipy as sp

#define colors
color_discrete_map= {'original': 'dimgray',
                     "word-order": "navy",
                     "information-loss": 'darkgreen',
                     "control": "lightgray"}

fig = px.scatter(plot_df, x="delta_surprisal", y="delta_brainscore",
                 trendline="ols", trendline_scope="overall",
                 template='simple_white',
                category_orders={"category": ['original','controls','information-loss', 'sentence-meaning','word-order']},
                labels={
                     "delta_surprisal": "Difference in average token surprisal to original sentences",
                     "delta_brainscore": "Difference in normalized brain predictivity <br>to original benchmark"},
                color="category",
                text='condition',
                color_discrete_map=color_discrete_map,
                width=780, height=500)

fig.update_layout(legend_title_text='Manipulation type')

r, p = sp.stats.pearsonr(plot_df['delta_surprisal'], plot_df['delta_brainscore'])
fig.update_layout(xaxis_range=[plot_df['delta_surprisal'].min()-0.2,plot_df['delta_surprisal'].max()+0.6])

annotation_x = plot_df['delta_surprisal'].min() + 0.5
annotation_y = plot_df['delta_brainscore'].max() + 0.05

fig.add_annotation(x=annotation_x, y=annotation_y,
            text='R={:.2f}, p={:.2g}'.format(r, p),
            showarrow=False,
            yshift=10, font=dict(size=16))
fig.update_traces(marker_size=15)
fig.update_traces(marker_symbol="square")
fig.update_layout(margin = {'l':0,'r':0,'t':0,'b':0})

def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    positions = ['middle right', 'middle left']
    return [positions[i % len(positions)] for i in range(len(x))]

fig.update_traces(textposition=improve_text_position(plot_df['condition']), textfont=dict(size=12))
# fig.update_layout(height=500, font=dict(size=12))
fig.update_xaxes(tickfont_size=16, ticklen=10, tickwidth=3,titlefont=dict(size=18))
fig.update_yaxes(tickfont_size=16, ticklen=10, tickwidth=3,titlefont=dict(size=18))


if save:
    fig.write_image(f"results_paper/surprisal2brainscore/surprisal2bs_{approach}.png", scale=1)
    fig.write_image(f"results_paper/surprisal2brainscore/surprisal2bs_{approach}.svg")

fig.show()

In [761]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# Sentence length 2 brain score

In [ ]:
conditions, categories, avg_sentence_length = [], [], []
for (cond, category) in CONDMAP:
    scr_perturb = COND2SET[cond]
    filename = f"{model_identifier}.{scr_perturb}+{cond}.sentence_surp.average_sentlength.txt"
    content = pd.read_csv(os.path.join(surprisal_path, filename), sep="\t")
    sentences = list(content["Sentence"])
    avg_lenth = np.mean([len(x.split()) for x in sentences])
    avg_sentence_length.append(avg_lenth)
    conditions.append(category)
    categories.append(scr_perturb)

length_df = pd.DataFrame({
    "condition" : conditions,
    "category" : categories,
    "avg_sentence_length" : avg_sentence_length
})

delta_length = [avg_sentence_length[i] - avg_sentence_length[0] for i in range(len(conditions))]

length_df["delta_length"] = delta_length
length_df

In [ ]:
#clean names
length_df['condition'] = length_df['condition'].str.replace("teston:","")
length_df['condition'] = length_df['condition'].replace(
{
'scr1': 'scrambled1',
'scr3': 'scrambled3',
'scr5': 'scrambled5',
'scr7': 'scrambled7'}
)

length_df = length_df.drop(columns=['category'])
length_df

In [ ]:
plot_df = length_df.merge(scores_df, how='inner', on='condition')
plot_df

In [ ]:
import plotly.express as px
import scipy as sp

#define colors
color_discrete_map= {'original': 'dimgray',
                     "word-order": "navy",
                     "information-loss": 'darkgreen',
                     "control": "lightgray"}

fig = px.scatter(plot_df, x="delta_length", y="delta_brainscore",
#                 size="error",
                 trendline="ols", template='simple_white',
                category_orders={"category": ['original','controls','information-loss', 'sentence-meaning','word-order']},
                labels={
                     "delta_length": "\u0394 sentence length",
                     "delta_brainscore": "\u0394 normalized predictivity"},
                color="category",
                text='condition',
                color_discrete_map=color_discrete_map,
                width=780, height=500)
                #symbol="category")
fig.update_layout(legend_title_text='Manipulation type')

r, p = sp.stats.pearsonr(plot_df['delta_length'], plot_df['delta_brainscore'])

fig.add_annotation(x=0, y=0.05,
            text='<b> Overall Pearson r={:.2f}, p={:.2g}'.format(r, p),
            showarrow=False,
            yshift=10)
fig.update_traces(marker_size=15)
fig.update_traces(marker_symbol="square")
fig.update_layout(margin = {'l':0,'r':0,'t':0,'b':0})

def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    positions = ['middle right']#, 'middle left']
    return [positions[i % len(positions)] for i in range(len(x))]

fig.update_traces(textposition=improve_text_position(plot_df['condition']))


# if save:
#     fig.write_image("results_paper/figure2.png", scale=1)
#     fig.write_image("results_paper/figure2.svg")

fig.show()